In [9]:
import time
import pandas as pd
from nba_api.stats.endpoints import leaguedashplayershotlocations
from functools import wraps

In [3]:
# Retry Wrapper 
def retry(max_attempts=3, delay=30):
  def decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
      attempts = 0
      while attempts < max_attempts:
        try:
          return func(*args, **kwargs)
        except Exception as e:
          print(f"Attempt {attempts + 1} failed: {e}")
          attempts += 1
          if attempts < max_attempts:
              print(f"Retrying in {delay} seconds...")
              time.sleep(delay)
      raise Exception(f"Function {func.__name__} failed after {max_attempts} attempts")
    return wrapper
  return decorator

In [4]:
from nba_api.stats.library.http import NBAStatsHTTP
import logging

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

class LoggedNBAStatsHTTP(NBAStatsHTTP):
    def send_api_request(self, endpoint, parameters, referer=None, proxy=None, headers=None, timeout=(20,60), raise_exception_on_error=False):
        logger.debug(f"Starting API request to {endpoint}")
        try:
            response = super().send_api_request(endpoint, parameters, referer, proxy, headers, timeout, raise_exception_on_error)
            logger.debug("Request completed successfully")
            logger.debug("Starting to process response")
            # Add any response info we can safely access
            return response
        except Exception as e:
            logger.error(f"Request failed with error: {str(e)}")
            logger.error(f"Error type: {type(e)}")
            raise

leaguedashplayershotlocations.NBAStatsHTTP = LoggedNBAStatsHTTP

In [8]:
import logging
import time
from nba_api.stats.library.http import NBAStatsHTTP
import requests

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

class LoggedNBAStatsHTTP(NBAStatsHTTP):
    def send_api_request(self, endpoint, parameters, referer=None, proxy=None, headers=None, timeout=60, raise_exception_on_error=False):
        logger.debug(f"API: Starting request to {endpoint}")
        logger.debug(f"API: Request parameters: {parameters}")
        
        start_time = time.time()
        
        try:
            # Get the full URL that would be used
            url = self.base_url + endpoint
            logger.debug(f"API: Full URL: {url}")
            
            # Create session with same settings
            session = requests.Session()
            if headers:
                session.headers.update(headers)
            
            # Make request with progress logging
            logger.debug("API: Initiating connection")
            response = session.get(url, params=parameters, timeout=timeout)
            logger.debug(f"API: Connected, status code: {response.status_code}")
            
            # Try to get content length
            content_length = response.headers.get('content-length')
            if content_length:
                logger.debug(f"API: Expected content length: {content_length} bytes")
            
            # Get actual content
            logger.debug("API: Starting to read content")
            content = response.content
            logger.debug(f"API: Received {len(content)} bytes")
            
            elapsed = time.time() - start_time
            logger.debug(f"API: Request completed in {elapsed:.2f} seconds")
            
            return response
            
        except Exception as e:
            elapsed = time.time() - start_time
            logger.error(f"API: Request failed after {elapsed:.2f} seconds")
            logger.error(f"API: Error type: {type(e)}")
            logger.error(f"API: Error details: {str(e)}")
            raise

leaguedashplayershotlocations.NBAStatsHTTP = LoggedNBAStatsHTTP

In [4]:
from nba_api.stats.static.teams import _find_team_by_abbreviation

_find_team_by_abbreviation("CLE")

{'id': 1610612739,
 'full_name': 'Cleveland Cavaliers',
 'abbreviation': 'CLE',
 'nickname': 'Cavaliers',
 'city': 'Cleveland',
 'state': 'Ohio',
 'year_founded': 1970}

In [5]:
def get_shooting_data():
  data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(timeout=(20,60))
  return data.shot_locations.get_data_frame()

def process_shooting_data(shootingDF):
  for fg in ['FGM', 'FGA']:
    shootingDF.loc[:, ('Total Shots', fg)] = shootingDF.xs(fg, level=1, axis=1).sum(axis=1) - shootingDF['Corner 3'][fg]
    shootingDF.loc[:, ('Total from 3', fg)] = shootingDF[['Above the Break 3', 'Corner 3', 'Backcourt']].xs(fg, level=1, axis=1).sum(axis=1)
    shootingDF.loc[:, ('Pct RA', fg)] = round(shootingDF['Restricted Area'][fg] / shootingDF['Total Shots'][fg], 3)
    shootingDF.loc[:, ('Pct 3', fg)] = round(shootingDF['Total from 3'][fg] / shootingDF['Total Shots'][fg], 3)
    shootingDF.loc[:, ('Pct Moreyball', fg)] = round(shootingDF['Pct RA'][fg] + shootingDF['Pct 3'][fg], 3)

  shootingDF = shootingDF[shootingDF['Total Shots']['FGA'] > 20].sort_values(by=('Pct Moreyball', 'FGA'), ascending=False)
  shootingDF.columns = [col[1] if col[0] == "" else '_'.join(col) for col in shootingDF.columns.values]

  return shootingDF

def save_to_csv(df, filename="Moreyball_Ranking_Test.csv"):
  df.to_csv(filename, index=False)

In [12]:
def main():
  try:
    shooting_df = get_shooting_data()
    #processed_df = process_shooting_data(shooting_df)
    return shooting_df
  except Exception as e:
    print(f"An error occurred: {e}")

In [15]:
get_shooting_data()

DEBUG:__main__:Starting API request to leaguedashplayershotlocations
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443
DEBUG:urllib3.connectionpool:https://stats.nba.com:443 "GET /stats/leaguedashplayershotlocations?College=&Conference=&Country=&DateFrom=&DateTo=&DistanceRange=By+Zone&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2024-25&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=&VsConference=&VsDivision=&Weight= HTTP/11" 200 28288
DEBUG:__main__:Request completed successfully
DEBUG:__main__:Starting to process response


ValueError: 2 columns passed, passed data had 30 columns

In [13]:
shooting_df = main()
#save_to_csv(processed_df)

DEBUG:__main__:Starting API request to leaguedashplayershotlocations
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443
DEBUG:urllib3.connectionpool:https://stats.nba.com:443 "GET /stats/leaguedashplayershotlocations?College=&Conference=&Country=&DateFrom=&DateTo=&DistanceRange=By+Zone&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2024-25&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=&VsConference=&VsDivision=&Weight= HTTP/11" 200 28288
DEBUG:__main__:Request completed successfully
DEBUG:__main__:Starting to process response


An error occurred: 2 columns passed, passed data had 30 columns


In [14]:
shooting_df

In [7]:
pip install pyyaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.2/183.2 kB 5.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [25]:
import yaml

In [24]:
processed_df = pd.read_csv('/Users/benporter/Documents/Github/nba-data/nba-data/data/dynamic/NBA_Leaders_MOREYBALL_Rate.csv')

In [27]:
processed_df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'AGE',
       'NICKNAME', 'Restricted Area_FGM', 'Restricted Area_FGA',
       'Restricted Area_FG_PCT', 'In The Paint (Non-RA)_FGM',
       'In The Paint (Non-RA)_FGA', 'In The Paint (Non-RA)_FG_PCT',
       'Mid-Range_FGM', 'Mid-Range_FGA', 'Mid-Range_FG_PCT',
       'Left Corner 3_FGM', 'Left Corner 3_FGA', 'Left Corner 3_FG_PCT',
       'Right Corner 3_FGM', 'Right Corner 3_FGA', 'Right Corner 3_FG_PCT',
       'Above the Break 3_FGM', 'Above the Break 3_FGA',
       'Above the Break 3_FG_PCT', 'Backcourt_FGM', 'Backcourt_FGA',
       'Backcourt_FG_PCT', 'Corner 3_FGM', 'Corner 3_FGA', 'Corner 3_FG_PCT',
       'Total Shots_FGM', 'Total from 3_FGM', 'Pct RA_FGM', 'Pct 3_FGM',
       'Pct Moreyball_FGM', 'Total Shots_FGA', 'Total from 3_FGA',
       'Pct RA_FGA', 'Pct 3_FGA', 'Pct Moreyball_FGA'],
      dtype='object')

In [30]:
mb_full_yaml_list =  [{
  'id': str(processed_df.iloc[i].PLAYER_ID),
  'name': str(processed_df.iloc[i].PLAYER_NAME),
  'team': str(processed_df.iloc[i].TEAM_ABBREVIATION),
  'RA_FGM': str(processed_df.iloc[i]['Restricted Area_FGM']),
  'RA_FGA': str(processed_df.iloc[i]['Restricted Area_FGA']),
  'RA_PCT': str(processed_df.iloc[i]['Pct RA_FGA']),
  '3PT_FGM': str(processed_df.iloc[i]['Total from 3_FGM']),
  '3PT_FGA': str(processed_df.iloc[i]['Total from 3_FGA']),
  '3PT_PCT': str(processed_df.iloc[i]['Pct 3_FGA']),
  'MB_FGM': str(processed_df.iloc[i]['Total from 3_FGM'] + processed_df.iloc[i]['Restricted Area_FGM']),
  'MB_FGA': str(processed_df.iloc[i]['Total from 3_FGA'] + processed_df.iloc[i]['Restricted Area_FGA']),
  'TOTAL_FGM': str(processed_df.iloc[i]['Total Shots_FGM']),
  'TOTAL_FGA': str(processed_df.iloc[i]['Total Shots_FGA']),
  'MB_PCT': str(processed_df.iloc[i]['Pct Moreyball_FGA'])
  } 
  for i in range((len(processed_df) - 1))]

with open('moreyball_full.yml', 'w') as stream:
  yaml.dump(mb_full_yaml_list, stream)

In [26]:
len(processed_df)

374